# Microplan Excel Validation Tool

Run all cells below (Shift+Enter) to start the validator.

In [1]:
# Cell 1: Install Requirements & Setup
import subprocess, sys, os, shutil

# Install from requirements.txt
print("Installing requirements from requirements.txt...")
if os.path.exists('requirements.txt'):
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("Requirements installed!")
        else:
            print("Some packages may have failed. Continuing anyway...")
            print(result.stderr[-500:] if result.stderr else "")
    except Exception as e:
        print(f"Warning: {e}. Continuing anyway...")
else:
    print("Warning: requirements.txt not found!")

# Imports
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from collections import defaultdict
import base64

# Import validator
from validator import Validator

# Clear and recreate folders
for folder in ['uploads', 'error']:
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder, exist_ok=True)

# Initialize validator
validator = Validator()

print("Setup complete!")

Installing requirements from requirements.txt...
Requirements installed!
Setup complete!


In [2]:
# Cell 3: Interactive UI

# ==================== STATE ====================
state = {
    'boundary_file': None,
    'facility_file': None,
    'boundary_sheets': [],
    'facility_sheets': [],
    'output_files': []
}

# ==================== WIDGETS ====================
out_status = widgets.Output()
out_results = widgets.Output()
out_downloads = widgets.Output()

mode = widgets.RadioButtons(
    options=['Single File (Excel with multiple sheets or CSV)',
             'Two Files (separate Boundary & Facility files - second file optional)'],
    value='Single File (Excel with multiple sheets or CSV)',
    description='', layout=widgets.Layout(width='100%')
)

upload1 = widgets.FileUpload(accept='.xlsx,.xls,.csv', multiple=False)
upload2 = widgets.FileUpload(accept='.xlsx,.xls,.csv', multiple=False)
label1 = widgets.HTML('<b>Upload File (Excel or CSV):</b>')
label2 = widgets.HTML('<b>Upload Facility File (Optional):</b>')
box2 = widgets.VBox([label2, upload2], layout=widgets.Layout(display='none'))

dd_boundary = widgets.Dropdown(options=['(auto-detect)'], value='(auto-detect)', description='Boundary Sheet:')
dd_facility = widgets.Dropdown(options=['(auto-detect)'], value='(auto-detect)', description='Facility Sheet:')
sheet_box = widgets.VBox([dd_boundary, dd_facility], layout=widgets.Layout(display='none'))

checks = {
    'non_zero_targets': widgets.Checkbox(value=True, description='Non-Zero Targets', indent=False),
    'naming_convention': widgets.Checkbox(value=True, description='Naming Convention', indent=False),
    'boundary_alignment': widgets.Checkbox(value=True, description='Boundary Alignment (requires both)', indent=False),
    'unique_names': widgets.Checkbox(value=True, description='Unique Names', indent=False),
    'user_mapping': widgets.Checkbox(value=True, description='User Mapping & Contacts', indent=False),
    'no_missing_entries': widgets.Checkbox(value=True, description='No Missing Entries', indent=False),
    'special_characters': widgets.Checkbox(value=True, description='Special Characters', indent=False),
}

btn_validate = widgets.Button(description='VALIDATE', button_style='primary', icon='check',
                               layout=widgets.Layout(width='150px', height='40px'))
btn_clear = widgets.Button(description='Clear', button_style='warning', icon='trash',
                            layout=widgets.Layout(width='100px', height='40px'))

# ==================== HELPER FUNCTIONS ====================

def show_status(msg, color='black'):
    with out_status:
        clear_output(wait=True)
        display(HTML(f'<p style="color:{color}; font-weight:bold; font-size:14px;">{msg}</p>'))

def is_csv(filename):
    return filename.lower().endswith('.csv')

def create_download_link(filepath):
    filename = os.path.basename(filepath)
    with open(filepath, 'rb') as f:
        data = f.read()
    b64 = base64.b64encode(data).decode()
    mime = 'text/csv' if filepath.endswith('.csv') else 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    
    return f'''
    <div style="margin:10px 0; padding:15px; background:#e8f5e9; border-radius:8px; border:2px solid #4caf50;">
        <div style="margin-bottom:10px;">
            <b style="color:#2e7d32;">Saved:</b> <code>{filepath}</code>
        </div>
        <a href="data:{mime};base64,{b64}" download="{filename}"
           style="display:inline-block; padding:12px 25px; background:#4caf50; color:white; 
                  text-decoration:none; border-radius:5px; font-weight:bold;">
            DOWNLOAD: {filename}
        </a>
    </div>'''

# ==================== EVENT HANDLERS ====================

def on_mode_change(change):
    if 'Two' in change['new']:
        box2.layout.display = 'block'
        label1.value = '<b>Upload Boundary File:</b>'
    else:
        box2.layout.display = 'none'
        label1.value = '<b>Upload File (Excel or CSV):</b>'

def save_upload(uploader, key):
    if not uploader.value:
        return
    
    files = uploader.value
    file_info = files[0] if isinstance(files, tuple) else list(files.values())[0]
    name = file_info.name if hasattr(file_info, 'name') else file_info['name']
    content = file_info.content if hasattr(file_info, 'content') else file_info['content']
    
    path = os.path.join('uploads', name)
    with open(path, 'wb') as f:
        f.write(content)
    state[key] = path
    
    try:
        if is_csv(name):
            sheet_box.layout.display = 'none'
            show_status(f'Loaded CSV: {name}', 'green')
        else:
            xls = pd.ExcelFile(path)
            sheets = ['(auto-detect)'] + xls.sheet_names
            if 'boundary' in key:
                dd_boundary.options = sheets
                if 'Single' in mode.value:
                    dd_facility.options = sheets
            else:
                dd_facility.options = sheets
            sheet_box.layout.display = 'block'
            show_status(f'Loaded Excel: {name} ({len(xls.sheet_names)} sheets)', 'green')
    except Exception as e:
        show_status(f'Error: {e}', 'red')

def on_upload1(change): save_upload(upload1, 'boundary_file')
def on_upload2(change): save_upload(upload2, 'facility_file')

def on_clear(b):
    state['boundary_file'] = None
    state['facility_file'] = None
    dd_boundary.options = dd_facility.options = ['(auto-detect)']
    sheet_box.layout.display = 'none'
    validator.reset()
    with out_results: clear_output()
    with out_downloads: clear_output()
    show_status('Cleared', 'orange')

def on_validate(b):
    validator.reset()
    
    # Collect files
    files = []
    if 'Two' in mode.value:
        if state['boundary_file']: files.append(state['boundary_file'])
        if state['facility_file']: files.append(state['facility_file'])
        if not files:
            show_status('Please upload at least one file!', 'red')
            return
    else:
        if not state['boundary_file']:
            show_status('Please upload a file first!', 'red')
            return
        files = [state['boundary_file']]
    
    # Set rules
    for k, cb in checks.items():
        validator.rules_enabled[k] = cb.value
    
    show_status('Validating...', 'blue')
    
    # Run validation
    b_sheet = None if dd_boundary.value == '(auto-detect)' else dd_boundary.value
    f_sheet = None if dd_facility.value == '(auto-detect)' else dd_facility.value
    
    all_issues = []
    total_summary = {'total': 0, 'errors': 0, 'warnings': 0, 'by_rule': defaultdict(int)}
    
    for fp in files:
        issues, summary = validator.validate_file(fp, b_sheet, f_sheet)
        all_issues.extend(issues)
        total_summary['total'] += summary['total']
        total_summary['errors'] += summary['errors']
        total_summary['warnings'] += summary['warnings']
        for r, c in summary['by_rule'].items():
            total_summary['by_rule'][r] += c
    
    # Cross-file alignment check
    if 'Two' in mode.value and state['boundary_file'] and state['facility_file'] and validator.rules_enabled['boundary_alignment']:
        try:
            b_df = pd.read_csv(state['boundary_file']) if is_csv(state['boundary_file']) else pd.read_excel(state['boundary_file'], sheet_name=b_sheet or 0)
            f_df = pd.read_csv(state['facility_file']) if is_csv(state['facility_file']) else pd.read_excel(state['facility_file'], sheet_name=f_sheet or 0)
            align_issues = validator.check_alignment(b_df, f_df, os.path.basename(state['boundary_file']), os.path.basename(state['facility_file']))
            all_issues.extend(align_issues)
            for i in align_issues:
                total_summary['total'] += 1
                total_summary['errors' if i['severity'] == 'error' else 'warnings'] += 1
                total_summary['by_rule'][i['rule']] += 1
        except:
            pass
    
    # Save and display
    output_files = validator.save_validated_files('error')
    display_results(all_issues, total_summary)
    display_downloads(output_files)
    show_status('Validation Complete!', 'green')

# ==================== DISPLAY FUNCTIONS ====================

def display_results(issues, summary):
    with out_results:
        clear_output(wait=True)
        color = '#27ae60' if summary['errors'] == 0 else '#e74c3c'
        status = 'All Passed!' if summary['errors'] == 0 and summary['warnings'] == 0 else ('Warnings Only' if summary['errors'] == 0 else 'Issues Found')
        pass_count, fail_count = validator.get_stats()
        
        html = f'''
        <div style="padding:15px; background:#f0f0f0; border-radius:8px; border-left:5px solid {color};">
            <h3 style="color:{color}; margin:0 0 10px 0;">{status}</h3>
            <div style="display:flex; gap:15px; flex-wrap:wrap;">
                <span style="padding:8px 15px; background:#27ae60; color:white; border-radius:4px;"><b>PASS:</b> {pass_count}</span>
                <span style="padding:8px 15px; background:#c0392b; color:white; border-radius:4px;"><b>FAIL:</b> {fail_count}</span>
                <span style="padding:8px 15px; background:#d68910; color:white; border-radius:4px;"><b>Warnings:</b> {summary['warnings']}</span>
            </div>
        </div>
        <h4>Issues by Rule:</h4><ul>'''
        
        for r, c in summary['by_rule'].items():
            html += f'<li><b>{r}:</b> {c}</li>'
        html += '</ul>' if summary['by_rule'] else '<p style="color:green;">No issues!</p>'
        
        if issues:
            html += '''<h4>Details (first 50):</h4>
            <div style="max-height:300px; overflow-y:auto;">
            <table style="width:100%; border-collapse:collapse; font-size:11px;">
            <tr style="background:#2c3e50; color:white;">
                <th style="padding:6px;">Sev</th><th>Rule</th><th>Sheet</th><th>Column</th><th>Row</th><th>Value</th><th>Message</th>
            </tr>'''
            for i in issues[:50]:
                c = '#c0392b' if i['severity'] == 'error' else '#d68910'
                html += f'''<tr>
                    <td style="padding:4px; border:1px solid #ddd; color:{c}; font-weight:bold;">{i['severity'][:3].upper()}</td>
                    <td style="border:1px solid #ddd;">{i['rule']}</td>
                    <td style="border:1px solid #ddd; font-size:10px;">{str(i['sheet'])[:25]}</td>
                    <td style="border:1px solid #ddd;">{str(i['column'])[:15]}</td>
                    <td style="border:1px solid #ddd;">{i['row']}</td>
                    <td style="border:1px solid #ddd;">{str(i['value'])[:20]}</td>
                    <td style="border:1px solid #ddd;">{i['message']}</td>
                </tr>'''
            html += '</table></div>'
        
        display(HTML(html))

def display_downloads(output_files):
    with out_downloads:
        clear_output(wait=True)
        if output_files:
            display(HTML('<h4 style="color:#1565c0;">Download Validated Files:</h4>'))
            for fp in output_files:
                display(HTML(create_download_link(fp)))

# ==================== CONNECT EVENTS ====================
mode.observe(on_mode_change, names='value')
upload1.observe(on_upload1, names='value')
upload2.observe(on_upload2, names='value')
btn_validate.on_click(on_validate)
btn_clear.on_click(on_clear)

# ==================== BUILD UI ====================
ui = widgets.VBox([
    widgets.HTML('<h2 style="color:#2c3e50; border-bottom:2px solid #3498db;">Microplan Validator</h2>'),
    widgets.HTML('<p style="color:#666;">Supports: .xlsx, .xls, .csv</p>'),
    widgets.HTML('<h4>Step 1: Select Mode</h4>'), mode,
    widgets.HTML('<h4>Step 2: Upload File(s)</h4>'), widgets.VBox([label1, upload1]), box2, out_status, sheet_box,
    widgets.HTML('<h4>Step 3: Validation Rules</h4>'), widgets.VBox(list(checks.values())),
    widgets.HTML('<h4>Step 4: Validate</h4>'), widgets.HBox([btn_validate, btn_clear]),
    widgets.HTML('<h4>Results:</h4>'), out_results,
    out_downloads
], layout=widgets.Layout(padding='15px'))

display(ui)